# Data Preprocessing
First import the data.

In [32]:
import numpy as np
X_test = np.load("project/X_test.npy")
y_test = np.load("project/y_test.npy")
person_train_valid = np.load("project/person_train_valid.npy")
X_train_valid = np.load("project/X_train_valid.npy")
y_train_valid = np.load("project/y_train_valid.npy")
person_test = np.load("project/person_test.npy")

In [33]:
# Ignore EOG data from last 3 of 25 electrodes
X_test = X_test[:, :-3, :]
X_test = X_test.transpose([0, 2, 1])
X_train_valid = X_train_valid[:, :-3, :]
X_train_valid = X_train_valid.transpose([0, 2, 1])

In [34]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 1000, 22)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [19]:
from sklearn import preprocessing

In [47]:
np.random.seed(0)

In [35]:
num_trials = X_train_valid.shape[0]
num_timesteps = X_train_valid.shape[1]
num_features = X_train_valid.shape[2]

In [48]:
rand_idx = np.random.permutation(num_trials)

In [50]:
rand_idx 

array([1122, 1491, 2081, ...,  763,  835, 1653])

In [58]:
# Create train and validation dataset split
split = int(num_trials * 0.80)
X_train = X_train_valid[rand_idx][:split]
y_train = y_train_valid[rand_idx][:split]
person_train = person_train_valid[rand_idx][:split]

X_valid = X_train_valid[rand_idx][split:]
y_valid = y_train_valid[rand_idx][split:]
person_valid = person_train_valid[rand_idx][split:]

In [60]:
person_valid.shape

(423, 1)